# Heading 

In [14]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
from IPython.core.display import display, HTML
import pandas as pd
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',100)    

display(HTML("<style>.container { width:95% !important; }</style>"))

In [15]:
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
from itertools import combinations, takewhile
import collections

from simhash import Simhash, SimhashIndex

sns.set()

print(sys.version)

3.6.8 |Anaconda custom (64-bit)| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]


In [16]:
print(spark.version)

2.4.0-cdh6.1.0


In [17]:
import pandas as pd
import numpy as np

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
# from pyspark.ml.feature import OneHotEncoderEstimator
# OneHotEncoderEstimator is available starting from Spark 2.3
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.sql.functions import *
from pyspark.sql.types import *

In [18]:
!hdfs dfs -ls -h '/user/ivy2/Tweets/' > '/home/sriharis/git_projects/BigDataEngg/final_project/file_list.txt'
tweets_path = '/user/ivy2/Tweets/'

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0


In [19]:
def file_read_from_head(fname, nlines):
        from itertools import islice
        with open(fname) as f:
                for line in islice(f, nlines):
                        print(line)
file_read_from_head('./file_list.txt',5)

Found 13642 items

-rw-r--r--   3 ivy2 ivy2     52.7 M 2017-06-22 10:15 /user/ivy2/Tweets/tweets201706221015.json

-rw-r--r--   3 ivy2 ivy2     50.0 M 2017-06-22 11:15 /user/ivy2/Tweets/tweets201706221115.json

-rw-r--r--   3 ivy2 ivy2     53.6 M 2017-06-22 12:15 /user/ivy2/Tweets/tweets201706221215.json

-rw-r--r--   3 ivy2 ivy2     54.4 M 2017-06-22 13:15 /user/ivy2/Tweets/tweets201706221315.json



#### Let's open a sample file and play with it for a while

In [20]:
df = spark.read.json('hdfs:///user/ivy2/Tweets/tweets201706221015.json')
df.cache()
df.count()

9973

In [23]:
df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |

In [41]:
fields_to_keep = ["id_str", "text", "truncated", "in_reply_to_status_id_str", "in_reply_to_user_id_str", "user.favourites_count", "user.id_str",
                  "coordinates", "quoted_status_id_str", "is_quote_status", "retweet_count", "favorite_count", "entities", "favorited", "retweeted",
                  "possibly_sensitive", "lang" ]

In [42]:
tweets_df = df.select(fields_to_keep).toPandas()

In [43]:
tweets_df.head()

,id_str,text,truncated,in_reply_to_status_id_str,in_reply_to_user_id_str,favourites_count,id_str,coordinates,quoted_status_id_str,is_quote_status,retweet_count,favorite_count,entities,favorited,retweeted,possibly_sensitive,lang
0,877892686513975296,RT @ArkansasBlog: Study: States with concealed carry laws experience rise in violent crime. http...,False,None,None,56574.0,155078285,None,None,False,0.0,0.0,"([], None, [], [(stanford.io/2sW8kYa, http://stanford.io/2sW8kYa, [92, 115], https://t.co/iPUk93...",False,False,False,en
1,877892684756566016,"IL Healthcare Truth Tour: Heal Don't Repeal! 6/22 2:30PM Federal Plaza, Chicago. Mobile billboar...",True,None,None,167.0,7159992,None,None,False,0.0,0.0,"([], None, [], [(twitter.com/i/web/status/8…, https://twitter.com/i/web/status/87789268475656601...",False,False,False,en
2,877892686744698882,"IL Healthcare Truth Tour: Heal Don't Repeal! 6/22 2:30PM Federal Plaza, Chicago. Mobile billboar...",True,None,None,111.0,827188692766834688,None,None,False,0.0,0.0,"([], None, [], [(twitter.com/i/web/status/8…, https://twitter.com/i/web/status/87789268674469888...",False,False,False,en
3,877892689118715904,"IL Healthcare Truth Tour: Heal Don't Repeal! 6/22 2:30PM Federal Plaza, Chicago. Mobile billboar...",True,None,None,1626.0,831319111414210560,None,None,False,0.0,0.0,"([], None, [], [(twitter.com/i/web/status/8…, https://twitter.com/i/web/status/87789268911871590...",False,False,False,en
4,877892690242936834,Day 1 of Chicago Summit: Gregory Touhill delivers keynote on Cybersecurity=Risk Management #ISMG...,False,None,None,119.0,39984598,None,None,False,0.0,0.0,"([([91, 103], ISMGSummits), ([104, 112], infosec)], [(pic.twitter.com/QTJYEkI10S, https://twitte...",False,False,False,en
